# Monitorização de Parâmetros Ambientais da Cidade de Lisboa

### Obter dados do Lisboa Aberta extraídos 

Manuel Banza - March 2021

In [2]:
import requests
from pandas.io.json import json_normalize
import pandas as pd

In [42]:
# Ligar ao JSON

url = 'http://opendata-cml.qart.pt:8080/lastmeasurements'
df = pd.read_json(url, dtype = {"date": int})
df['date'] = df['date'].astype(str)
df['date_modified'] = df['date'].map(lambda x: x[:-6])
df['date_modified'] = pd.to_datetime(df['date_modified'], format='%Y%m%d%H%M')
df.head()

,id,date,dateStandard,value,unit,address,coordinates,date_modified
0,QA00NO0001,202103312300000000,UTC,41.0,µg/m3,CALÇADA DA AJUDA,"{'x': -92781, 'y': -106663, 'z': 0, 'lat': 38....",2021-03-31 23:00:00
1,QA0NO20001,202103312300000000,UTC,141.0,µg/m3,CALÇADA DA AJUDA,"{'x': -92781, 'y': -106663, 'z': 0, 'lat': 38....",2021-03-31 23:00:00
2,QA00O30001,202103312300000000,UTC,129.0,µg/m3,CALÇADA DA AJUDA,"{'x': -92781, 'y': -106663, 'z': 0, 'lat': 38....",2021-03-31 23:00:00
3,QAPM250001,202103310000000000,UTC,17.0,µg/m3,CALÇADA DA AJUDA,"{'x': -92781, 'y': -106663, 'z': 0, 'lat': 38....",2021-03-31 00:00:00
4,QAPM100001,202103310000000000,UTC,44.0,µg/m3,CALÇADA DA AJUDA,"{'x': -92781, 'y': -106663, 'z': 0, 'lat': 38....",2021-03-31 00:00:00


In [43]:
df['acronimo_short'] = df['id'].map(lambda x: x[:4])

In [44]:
df.acronimo_short.value_counts()

ME00    79
QAPM    56
QA00    50
QA0N    28
METE    26
RULA    20
QA0S     9
Name: acronimo_short, dtype: int64

In [45]:
df.unit.value_counts()

µg/m3    143
%         26
Mbar      26
ºC        26
dB(A)     20
º          9
mm         9
km/h       9
Name: unit, dtype: int64

In [46]:
df.date_modified.value_counts()

2021-03-31 23:00:00    180
2021-03-31 00:00:00     65
2021-03-31 16:00:00     23
Name: date_modified, dtype: int64

In [47]:
df['date_modified'].describe()

<ipython-input-47-8dd75c4a617b>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df['date_modified'].describe()


count                     268
unique                      3
top       2021-03-31 23:00:00
freq                      180
first     2021-03-31 00:00:00
last      2021-03-31 23:00:00
Name: date_modified, dtype: object

In [ ]:
# Criar nova coluna baseado em multiplas condições       
def unit_topic(x):
  if x['unit'] == 'µg/m3': return 'QUALIDADE DO AR'
  elif x['unit'] == 'dB(A)': return 'RUÍDO'
  elif x['unit'] == '%': return 'Humidade relativa'
  else: return 'N/A'
  
df['unit_topic'] = df.apply(unit_marcacao, axis=1)

In [ ]:
# Criar nova coluna baseado em multiplas condições       
def unit_topic(x):
  if x['unit'] == 'µg/m3': return 'QUALIDADE DO AR:'
  elif x['unit'] == '%': return 'Humidade relativa'
  elif x['unit'] == 'Mbar': return 'Pressão atmosférica'
  elif x['unit'] == 'ºC': return 'Temperatura'
  elif x['unit'] == 'dB(A)': return 'Nível sonoro contínuo equivalente (LAEQ)'
  elif x['unit'] == 'º': return 'Vento direção '
  elif x['unit'] == 'mm': return 'Precipitação'
  elif x['unit'] == 'km/h': return 'Vento intensidade'
  else: return 'N/A'
  
df['unit_topic'] = df.apply(unit_marcacao, axis=1)

In [41]:
# Get today date now to file name when export to csv or excel with encoding utf8
from datetime import datetime
df.to_csv(datetime.now().strftime('../data_sources/output/parametros_ambientais-%Y-%m-%d-%H-%M-%S.csv'), encoding='utf8')